## Reinforcement Learning Tutorial -1: Q Learning

#### MD Muhaimin Rahman
sezan92[at]gmail[dot]com

Q learning , can be said one of the most famous -and kind of intuitive- of all Reinforcement learning algorithms. In fact ,the recent all algorithms using Deep learning , are based on the Q learning algorithms. So, to work on recent algorithms, one must have a good idea on Q learning.

### Intuition

First , start with an Intuition. Lets assume , you are in a maze

![](QLearningGame.gif)

Okay okay! I admit, it is not a maze. just a house with 5 rooms. And I got it from, this [link](http://mnemstudio.org/path-finding-q-learning-tutorial.htm) . Your goal is to get out of this place, no matter where you are. But you dont know - atleast pretend to - how to get there! After wondering about the map, you stumbled upon a mysterious letter with a lot of numbers in the room.

![QMatrix](q_matrix5.gif)

The matrix has 6 columns and 6 rows. What you will have to do , is to go to the room with highest value. Suppose, you are in room number 2. Then , you will have to move to room number 3 . Then you get out! Look at the picture again! You can try with every state, you are guaranteed to get out of the house, using this matrix! .


In the world of RL, every room is called a ```state```, movement from one state to another is called ```action```. Our game has a very ***JARGONISH*** name, ```Markov Decision Process``` . Maybe they invented this name to freak everybody out. But in short, this process means, your action from current state never depends on previous state. Practically such processes are impossible, but it helps to simplify problems 

 Now the question is , how can we get this ? 

- First , initialize the matrix as Zeros

![](q_matrix1.gif)

- Then we will apply the Q learning update equation

\begin{equation}
Q(s_t,a) = Q(s_t,a) + \alpha (Q'(s_{t+1},a)-Q(s_t,a))
\end{equation}

Here, $s_t$ is state at time $t$ , $s_{t+1}$ means the next state, $a$ is action. Q(s_t,a_t) means Q matrix value for state $s_t$ and action $a_t$ , $Q'(s_{t+1},a)$ means target Q value with state $s_{t+1}$ and the ***BEST ACTION*** for next state. Here $\alpha $ is learning rate}

Before we proceed, let me ask you, does this equation ring a bell ? I mean, haven't you seen a similar equation ? 

Yeah, you got it , it is similar to Gradient descent Equation. If you dont know Gradient descent equation, I am sorry, you wont be able to get the future tutorials. So I suggest you get the basic and working Idea of Neural Networks and Gradient descent algorithms

Now ,How can we get $Q'(s_{t+1},a)$ ? 

Using Bellman Equation

\begin{equation}
Q'(s_{t+1},a) = r+ \gamma max(Q(s_{t+1},a_t))
\end{equation}

Here, $r$ is reward we get-if we can get - from one state to another state.

It means the target $Q$ value for every state and action is the sum of reward with that state and action, and the maximum $Q$ value of next state multiplied with discount factor $\gamma$. You can get the idea in the later section

***Where did this equation came from ? ***

Okay chill! let's start from the game again ! So suppose , every room has reward, $R_t,R_{t+1},R_{t+2},R_{t+3},R_{t+4},R_{t+5}$.. So obviously , the value of a state will be the expected cumulative reward

\begin{equation}
Q(s,a) = R_t + R_{t+1} + R_{t+2}+ R_{t+3}+ R_{t+4}+ R_{t+5}
\end{equation}

Suppose, someone comes here, and says, He wants give more weight to sooner rewards than later rewards. What should we do ? We will introduce, discount factor, $\gamma$ , which is $0<\gamma<1$ ..

\begin{equation}
Q(s,a) = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2}+ \gamma^3 R_{t+3}+ \gamma^4 R_{t+4}+ \gamma^5 R_{t+5}
\end{equation}

\begin{equation}
Q(s,a) = R_t + \gamma [R_{t+1} + \gamma R_{t+2}+ \gamma^2 R_{t+3}+ \gamma^3 R_{t+4}+ \gamma^4 R_{t+5}]
\end{equation}

This equation can be rewritten as

\begin{equation}
Q(s_t,a) = R_t+\gamma Q(s_{t+1},a_{t+1})
\end{equation}

Suppose, we have some finite discrete actions in our hand, and each resulting $Q$ values of its own, what we will do ? We will try to take the action of maximum $Q$ value!

\begin{equation}
Q(s_t,a) = R_t+\gamma max(Q(s_{t+1},a))
\end{equation}

### Coding!

Let's start coding!

I will be using ***Open Ai*** gym environment. The Introduction and Installtion of environments are given [here](https://github.com/openai/gym)

In [ ]:
import gym
import numpy as np

Initialization of Environments

I will use the Mountaincar environment by Open AI gym. It is a classic problem invented from 90s. I intend to use this environment for all algorithms .

![](MC.jpg)

In this game, your task is to get the car reach that green flag. For every step you will get -1 .So , your job is to reach the goal position with minimum steps. Maximum steps limit is 200. 

In [ ]:
env = gym.make('MountainCar-v0')
s = env.reset() #Reset the car

```env.reset()``` gives the initial state. State is the position and velocity of the car in a given time

This game's actions can be 0,1,2 . 0 for left, 1 for doing nothing, 2 for right
```env.step(action)``` returns four arguments

- next state
- reward
- terminal , it means if game is over or not
- info , for now , it is unnecessary

#### Hyper Parameters

- ```legal_actions``` number of actions
- ```actions``` the actions list
- ```gamma``` discount factor $\gamma$
- ```lr``` learning rate $\alpha$
- ```num_episodes``` number of episodes
- ```epsilon``` epsilon , to choose random actions 
- ```epsilon_decay``` epsilon decay rate

In [ ]:
legal_actions=env.action_space.n
actions = [0,1,2]
gamma =0.99
lr =0.5
num_episodes =30000
epsilon =0.5
epsilon_decay =0.99

Codeblock to discretize the state. Because ***Q learning*** doesnt work on continuous state space, we have to convert states into 10 discrete states

In [ ]:
N_BINS = [10,10]

MIN_VALUES = [0.6,0.07]
MAX_VALUES = [-1.2,-.07]
BINS = [np.linspace(MIN_VALUES[i], MAX_VALUES[i], N_BINS[i]) for i in range(len(N_BINS))]
rList =[]
def discretize(obs):
       return tuple([int(np.digitize(obs[i], BINS[i])) for i in range(len(N_BINS))])

#### Q Learning CLass

In [ ]:
class QL:
    def __init__(self,Q,policy,
                legal_actions,
                actions,
                gamma,
                lr):
        self.Q = Q #Q matrix
        self.policy =policy
        self.legal_actions=legal_actions
        self.actions = actions
        self.gamma =gamma
        self.lr =lr
    def q_value(self,s,a):
        """Gets the Q value for a certain state and action"""
        if (s,a) in self.Q:
            self.Q[(s,a)]
        else:
            self.Q[s,a]=0
        return self.Q[s,a]
    def action(self,s):
        """Gets the action for cetain state"""
        if s in self.policy:
            return self.policy[s]
        else:
            self.policy[s] = np.random.randint(0,self.legal_actions)
            return self.policy[s]
    def learn(self,s,a,s1,r,done):
        """Updates the Q matrix"""
        if done== False:
            self.Q[(s,a)] =self.q_value(s,a)+ self.lr*(r+self.gamma*max([self.q_value(s1,a1) for a1 in self.actions]) - self.q_value(s,a))
        else:
            self.Q[(s,a)] =self.q_value(s,a)+ self.lr*(r - self.q_value(s,a))
        self.q_values = [self.q_value(s,a1) for a1 in self.actions]
        self.policy[s] = self.actions[self.q_values.index(max(self.q_values))]


#### Q Matrix Parameters

- ```Q``` - Q table. We will use dictionary data structure.
- ```policy``` - policy table , it will give us the action for given state


In [ ]:
Q = {}
policy ={}
legal_actions =3
QL = QL(Q,policy,legal_actions,actions,gamma,lr)

#### Training

### Psuedocode

- get initial state $s_{raw}$
- discretize initial state , $s \gets discretize(s_{raw})$
- set total reward to zero , $r_{total} \gets 0$
- set terminal $d$ to false , $d \gets False$
- for each step
- - choose action based on epsilon greedy policy
- - get next state $s1_{raw} $, reward , $r$, terminal $d$ doing the action
- - $s1 \gets discretize(s1_{raw}) $
- - $r_{total} \gets r_{total}+r$
- - if $d == True $ 
- - - if $r_{total}<-199$ 
- - - - then give $r \gets -100$
- - - - Update $Q$ table
- - - - break 
- - else 
- - - Update $Q$ table
- - - break
- - $s \gets s1$

In [ ]:
for i in range(num_episodes):
    s_raw= env.reset() #initialize
    s = discretize(s_raw) #discretize the state
    rAll =0 #total reward
    d = False
    j = 0
    for j in range(200):
        
        #epsilon greedy. to choose random actions initially when Q is all zeros
        if np.random.random()< epsilon:
            a = np.random.randint(0,legal_actions)
            epsilon = epsilon*epsilon_decay
        else:
            a =QL.action(s)
        s1_raw,r,d,_ = env.step(a)
        rAll=rAll+r
        s1 = discretize(s1_raw)
        env.render()
        if d:
            if rAll<-199:
                r =-100 #punishment, if the game finishes before reaching the goal , we can give punishment
                QL.learn(s,a,s1,r,d)
                print("Failed! Reward %d"%rAll)
            elif rAll>-199:
                print("Passed! Reward %d"%rAll)
            break
        QL.learn(s,a,s1,r,d)
        if j==199:
            print("Reward %d after full episode"%(rAll))
            
        s = s1
env.close()  